In [1]:
import gym

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 26 23:20:19 2018

@author: jpmaldonado
"""

import numpy as np

solution = np.array([1, 1, -0.5])
# the function we want to optimize for sanity check
def f(theta):
  # Here would go the evaluation of the episode
  reward = -np.sum(np.square(solution - theta))
  return reward

In [ ]:
#################################
# STARTER CODE - CEM
#################################
dim_theta = 3
theta_mean = np.zeros(dim_theta)
theta_std = np.ones(dim_theta)
batch_size = 25 # number of samples per batch
elite_frac = 0.2 # fraction of samples used as elite set


n_iter = 300
# Now, for the algorithms
for _ in range(n_iter):

    # Sample parameter vectors    

    #YOUR CODE HERE    

    # Get elite parameters
    
    #YOUR CODE HERE
    
    # Update theta_mean, theta_std
    
    #YOUR CODE HERE

In [ ]:
#################################
# STARTER CODE - NES
#################################
w0 = np.random.randn(3) #initial guess
npop=50
n_iter=300
sigma=0.1
alpha=0.001

for _ in range(n_iter):
    # Sample vectors from a normal distribution
    
    # YOUR CODE HERE
    
    # Sample function values by evaluating on the population
    
    # YOUR CODE HERE
    
    # Optional: standardize (substract mean and divide by std)
    
    # "Gradient" update
    
    # YOUR CODE HERE
